In [ ]:
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install selenium=3.141

# IMPORTANDO BIBLIOTECAS E FUNÇÕES AUXILIARES

In [1]:
from datetime import timedelta
import pandas as pd
import numpy as np
import cv2 as cv
import json
import os

from Modulos.BROWSER.Engine import *
from Modulos.DADOS.Engine import *
from Functions import *
from Script1 import *
from Default import *

# ABRINDO O DADO DOS VENTOS
DataFilesAuto = ReadDatasets(WeatherStationsPath, SaveAnalysis=WeatherStationsPath_OK)

# ABRINDO NAVEGADOR PARA OBTER A DECLINAÇÃO MAGNETICA
driver = OpenBrowser(url=Url_MagneticDeclination)


# BUSCANDO RUMO QUE MÁXIMIZA O FO

In [69]:
# PERCORRENDO OS DADOS DE CADA AEROPORTOE OTIMIZANDO VALORES
FinalResult = []
aux_aeroporto = ""
for aeroporto, ideaeroporto in zip(DataFilesAuto, range(1,len(list(DataFilesAuto.keys()))+1)):

    print(f"AEROPORTO: {aeroporto} - #/$".replace("#",str(ideaeroporto)).replace("$",str(len(list(DataFilesAuto)))))
    
    # OBTENDO OS DADOS DO AEROPORTO
    Info = DataFilesAuto[aeroporto]
    tabelao = Info["Dataset"].copy()
    local = Info["Local"]
    
    # FILTRANDO DADOS PARA ANALISES P/5, 10, 15 e 20 ANOS
    tabelao5    = tabelao[tabelao["DATA"]>=tabelao["DATA"].max()-timedelta(days=365*5 )].reset_index(drop=True).copy()
    tabelao10   = tabelao[tabelao["DATA"]>=tabelao["DATA"].max()-timedelta(days=365*10)].reset_index(drop=True).copy()
    tabelao15   = tabelao[tabelao["DATA"]>=tabelao["DATA"].max()-timedelta(days=365*15)].reset_index(drop=True).copy()
    tabelao20   = tabelao[tabelao["DATA"]>=tabelao["DATA"].max()-timedelta(days=365*20)].reset_index(drop=True).copy()
    
    # OBTENDO A DECLINAÇÃO MAGNETICA
    if (aux_aeroporto != aeroporto) is True:
        aux_aeroporto = aeroporto 
        MagneticDeclination = round(GetMagneticDeclination(local[0], local[1], driver), DecimalPlaces)
    
    # DESENHANDO A ROSA DOS VENTOS
    if True:
        
        # IMAGEM EM BRANCO
        imagem_branca = np.zeros((Height_IMG, Width_IMG, 3), dtype=np.uint8)

        # Calculando as coordenadas do centro da imagem
        centro_x = int(Width_IMG  / 2)
        centro_y = int(Height_IMG / 2)

        maior = max(LIMITES)
        menor = min(LIMITES)

        proporcao = ProportionWindRoseImg/maior
        comprimento = int(Width_IMG * (maior * proporcao))
        WidthRunway = 0
        for limite in LIMITES[::-1]:
            r = int(Width_IMG * (limite * proporcao))
            if limite == LIMITES[-1]: cv.circle(imagem_branca, (centro_x, centro_y), r, (255, 255, 255), -1)
            cv.circle(imagem_branca, (centro_x, centro_y), r, (0, 0, 255), 2)
            if limite == WindRunwayLimite: WidthRunway = r

        # Desenhando as linhas radiais para cada direção
        graus_rumo = calcular_setores(RoseWind, SectorNames[RoseWind])
        for name, angulo in graus_rumo.items(): DrawRadialLine(imagem_branca, (centro_x, centro_y), comprimento, angulo[0], (0, 0, 0), 2)

    # IDENTIFICANDO CADA AREA DA ROSA DOS VENTOS COM UMA COR ESPECIFICA
    if True:
        imagem_AREA = imagem_branca.copy()

        # Crie uma máscara para identificar as regiões brancas na imagem
        branco_baixo = np.array([255, 255, 255])
        branco_alto = np.array([255, 255, 255])
        mascara_branco = cv.inRange(imagem_AREA, branco_baixo, branco_alto)

        # Encontre os contornos na imagem mascarada
        contornos, _ = cv.findContours(mascara_branco, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)

        # REALIZANDO O AGRUPAMENTO DAS AREAS
        rotulos_clusters, grupos, center_grups = Agroup(contornos)

        # Criar uma nova imagem para desenhar os contornos coloridos
        cores_grupos = GenerateUniqueGrayColors(len(LIMITES)*RoseWind)

        # SALVANDO INFORMAÇÕES SOBRE AS AREAS
        dict_area_info = {}

        # Para cada grupo de contornos, criar uma paleta de cores única
        contagem = 0
        
        # ORDENANDO DOS MAIORES GRUPOS PARA OS MENORES 
        grupos = sorted(grupos, key=lambda x: cv.contourArea(x[0]), reverse=True)
        id_grupo = 1
        for grupo, limite in zip(grupos, LIMITES[::-1]):
            # Gerar cores únicas para o grupo com base no número de contornos
            parcial = {}
            for contorno in grupo:
                cor = cores_grupos[contagem]  # Selecionar uma cor única da paleta para cada área dentro do grupo
                cv.drawContours(imagem_AREA, [contorno], -1, cor, thickness=cv.FILLED)  # Preencher o contorno com a cor
        
                # # CALCULANDO ANGULO DO CENTRO PARA AREA PARA IDENTIFICAR O RUMO
                p1 = BaricentroArea(contorno)
                p2 = centro_x, centro_y
                angulo = CalculateAzimuth(p1, p2)
                rumo = ""
                for name, angulos in graus_rumo.items():
                    maior_ = max(angulos)
                    menor_ = min(angulos)
                    if angulo >= menor_ and angulo <= maior_:
                        rumo = name
                    elif angulo <= menor_ and angulo <= maior_:
                        rumo = "N"

                contagem+=1
                
                parcial[rumo] = {
                    # "CONTORNO": contorno, 
                    "AREA": cv.contourArea(contorno), 
                    "COR": cor,
                    "LIMITE": limite
                }
            dict_area_info[id_grupo] = parcial
            id_grupo+=1
    
    # DESENHANDO O RETANGULO DA PISTA E RECORTANDO
    if True:

        # Definindo as dimensões do retângulo
        largura_retangulo = WidthRunway * 2
        altura_retangulo = comprimento * 2

        # Calculando os vértices do retângulo
        vertices = np.array([
            [centro_x - largura_retangulo / 2, centro_y - altura_retangulo / 2],
            [centro_x + largura_retangulo / 2, centro_y - altura_retangulo / 2],
            [centro_x + largura_retangulo / 2, centro_y + altura_retangulo / 2],
            [centro_x - largura_retangulo / 2, centro_y + altura_retangulo / 2]
        ], dtype=np.float32)

        # CRIANDO PONTO DE RESTAURAÇÃO DA IMG SEM DESENHO
        imagem_AREA_BK = imagem_AREA.copy()
        
        # PERCORRENDO DADOS
        DFS = {
            ">=5"   :tabelao5,
            ">=10"  :tabelao10,
            ">=15"  :tabelao15,
            ">=20"  :tabelao20
        }
        dict_final = {}
        dict_pacial = {}
        for Name in DFS:
            Dataset = DFS[Name]
            
            # LIMPANDO A PASTA DE IMGS SE FOR FAZER UM NOVO VIDEO
            if MakeVideo is True: ClearFolder(FolderImages)
            
            # SE EXISTIR DADOS PROSSIGA
            if len(Dataset) > 0:
            
                # CONTABILIZANDO VENTOS PARA CADA DIRECAO EM PORCENTAGEM
                df_pct_ventos = Script1(Dataset[DirectionName], Dataset[WindName], SectorNames[RoseWind], LIMITES)
                
                # DADOS PARA SALVAR A OTIMIZAÇÃO
                FU_FINAL = 0
                FU_FINAL_MAXIMO = 0
                FU_FINAL_GRAU = 0
                
                # ANGULANDO A PISTA PARA BUSCAR OTIMIZAR O FU
                for grau in range(0, MaxSpinRoseWind+1):
                    
                    # RESTAURANDO BK DA IMG SEM DESENHOS
                    imagem_AREA = imagem_AREA_BK.copy()
                    
                    # Criando uma matriz de rotação de 30 graus em torno do centro da imagem
                    theta = np.radians(grau)
                    rot_mat = np.array([[np.cos(theta), -np.sin(theta)], [np.sin(theta), np.cos(theta)]])
                    centro_retangulo = np.array([centro_x, centro_y])
                    vertices_rotacionados = np.dot(vertices - centro_retangulo, rot_mat.T) + centro_retangulo

                    # Desenhando o retângulo na imagem
                    cv.polylines(
                        imagem_AREA, 
                        [vertices_rotacionados.astype(np.int32)], 
                        isClosed=True, 
                        color=ColorRunWay, 
                        thickness=2
                    )
                    
                    # Crie uma máscara para identificar as regiões brancas na imagem
                    mascara = cv.inRange(
                        imagem_AREA, 
                        ColorRunWay, 
                        ColorRunWay
                    )

                    # Encontre os contornos na imagem mascarada
                    contornos, _ = cv.findContours(
                        mascara, 
                        cv.RETR_EXTERNAL, 
                        cv.CHAIN_APPROX_NONE
                    )
                    
                    # PINTANDO A PISTA COMPLETA DE UMA COR PARA IDENTIFICAR POR MASCARA E RECORTAR
                    imagem_AREA2 = imagem_AREA_BK.copy()
                    cv.drawContours(
                        imagem_AREA2, 
                        contornos, 
                        -1, 
                        ColorRunWay,
                        thickness=cv.FILLED
                    )
                    
                    # Aplicar a máscara na imagem original
                    mascara_recorte = cv.inRange(
                        imagem_AREA2, 
                        ColorRunWay,
                        ColorRunWay
                    )
                    pista_recortada = cv.bitwise_and(
                        imagem_AREA, 
                        imagem_AREA,
                        mask=mascara_recorte
                    )
                    
                    # PUXANDO TABELA DE VENTOS DO SCRIPT 1 E RENOMEANDO AS ValoresS 
                    # PARA UMA CONSULTA MAIS FACILITADA
                    df_pct_ventos_cp = df_pct_ventos.copy()
                    df_pct_ventos_cp.columns = LIMITES + ["40+"]

                    # FIXANDO A PISTA DE MAIOR THETA
                    if FU_FINAL > FU_FINAL_MAXIMO:
                        FU_FINAL_MAXIMO = FU_FINAL
                        FU_FINAL_GRAU = grau
                    
                    # SE EXISTIR UM FU DESENHE 
                    if FU_FINAL_MAXIMO > 0:
                                
                        # Criando uma matriz de rotação de 30 graus em torno do centro da imagem
                        theta = np.radians(FU_FINAL_GRAU)
                        rot_mat = np.array([[np.cos(theta), -np.sin(theta)], [np.sin(theta), np.cos(theta)]])
                        centro_retangulo = np.array([centro_x, centro_y])
                        vertices_rotacionados = np.dot(vertices - centro_retangulo, rot_mat.T) + centro_retangulo
                        
                        # Desenhando o retângulo na imagem
                        cv.polylines(
                            imagem_AREA, 
                            [vertices_rotacionados.astype(np.int32)], 
                            isClosed=True, 
                            color=ColorBestRunWay, 
                            thickness=2
                        )

                        # DESENHANDO REFERENCIA DOS DADOS
                        DrawReferenceRUNWAY(
                            imagem_AREA, 
                            (centro_x, centro_y), 
                            comprimento, 
                            FU_FINAL_GRAU, 
                            (255, 165, 0), 
                            25
                        )
                    
                    # ANALISANDO O FATOR DE ULILIZAÇÃO DA PISTA ATUAL
                    FU_FINAL = 0
                    for id_grupo in dict_area_info:
                        grupo = dict_area_info[id_grupo]
                        for rumo in grupo:
                            Info = grupo[rumo]
                            cor_grupo = Info["COR"]
                            limite = Info["LIMITE"]
                            
                            # Crie uma máscara para identificar as regiões brancas na imagem
                            mascara = cv.inRange(
                                pista_recortada.copy(), 
                                cor_grupo, 
                                cor_grupo
                            )
                            
                            # Encontre os contornos na imagem mascarada
                            contorno, _ = cv.findContours(
                                mascara, 
                                cv.RETR_EXTERNAL, 
                                cv.CHAIN_APPROX_NONE
                            )
                            contorno = list(contorno)
                            
                            # NEM TODA AERA ESTA DENTRO DA PISTA LOGO 
                            if len(contorno) > 0:  area_dentro_PPD = cv.contourArea(contorno[0])
                            # CASO NÃO ESTIVER SETE ZERO
                            else: area_dentro_PPD = 0
                            
                            # FAZENDO A FRAÇÃO DOS VENTROS DENTRO DA PISTA
                            fracao_dentro = ( area_dentro_PPD / Info["AREA"] ) * df_pct_ventos_cp.loc[rumo][limite]
                            
                            FU_FINAL = FU_FINAL + fracao_dentro
                    FU_FINAL = round(FU_FINAL, DecimalPlaces)
                    
                    # INFORMAÇÕES SOBRE A PISTA ATUAL
                    texts = [
                        f'DIRECTION NOW',
                        f'FO: {FU_FINAL}%',
                        f'RUMO: {grau}',
                        f'MAGNETIC DECLINATION: {MagneticDeclination}',
                        f'RUNWAY ORIENTATION: {HeadboardRunway(round(grau - MagneticDeclination, DecimalPlaces))}',
                        f'CROSS WIND: {round(100 - FU_FINAL, DecimalPlaces)}%',
                        
                    ]
                    for text, id in zip(texts, list(range(1, len(texts)+1))):
                        cv.putText(imagem_AREA, text, (StartLegendRight, HeightEspaceLegend*id), Fonte, FonteSize, ColorRunWay, FonteThickness, cv.LINE_AA)

                    # INFORMAÇÕES SOBRE A MELHOR PISTA
                    if FU_FINAL_MAXIMO > 0:
                        
                        texts = [
                            f'BEST DIRECTION',
                            f'FO: {FU_FINAL_MAXIMO}%',
                            f'RUMO: {FU_FINAL_GRAU}',
                            f'MAGNETIC DECLINATION: {MagneticDeclination}',
                            f'RUNWAY ORIENTATION: {HeadboardRunway(round(FU_FINAL_GRAU - MagneticDeclination, DecimalPlaces))}',
                            f'CROSS WIND: {round(100 - FU_FINAL_MAXIMO, DecimalPlaces)}%',
                        ]
                        for text, id in zip(texts, list(range(1, len(texts)+1))):
                            cv.putText(imagem_AREA, text, (StartLegendLeft, HeightEspaceLegend*id), Fonte, FonteSize, ColorBestRunWay, FonteThickness, cv.LINE_AA)

                    # DESENHANDO PARTE INFERIOR DA IMAGEM
                    texts = [
                        f'Airport: {aeroporto}',
                        f'Lat: {local[0]} / Lon: {local[1]}'
                    ]
                    texts.reverse()
                    for text, id in zip(texts, list(range(1, len(texts)+1))):
                        cv.putText(imagem_AREA, text, (StartLegendLeft, Height_IMG-HeightEspaceLegend*id), Fonte, FonteSize, ColorLegend_BottomLeft, FonteThickness, cv.LINE_AA)

                    # DESENHANDO A REFERENCIA DA PISTA
                    DrawReferenceRUNWAY(imagem_AREA, (centro_x, centro_y), comprimento, grau, ColorPointRef, PointSizeRef)
                    
                    # SE DESEJAR CRIAR UM VIDEO DA SIMULAÇÃO SALVE AS IMGS
                    if MakeVideo is True:
                        
                        # SALVANDO IMAGEM PARA O VIDEO FINAL
                        cv.imwrite(os.path.join(FolderImages, f"IMG{grau}.jpg"), imagem_AREA)

                dict_pacial[Name] = {
                    "FO": FU_FINAL_MAXIMO,
                    "RUMO": FU_FINAL_GRAU,
                    "CROSS WIND": round(100 - FU_FINAL_MAXIMO, DecimalPlaces)
                }
                
                # SE ATIVADO CRIA O VIDEO
                numbers = "".join([row for row in Name if row.isdigit()])
                if MakeVideo is True: 
                    path_save = caminho_saida_video.format(aeroporto, numbers)
                    if os.path.exists(os.path.dirname(path_save)) is False:
                        os.makedirs(os.path.dirname(path_save))
                    CreateVideo(FolderImages, path_save)

        dict_final[aeroporto] = dict_pacial
        FinalResult.append(dict_final)
        with open("FinalResult.json", "w", encoding="UTF-8") as file:
            json.dump(FinalResult, file)


AEROPORTO: JOAO PESSOA - 1/1
